## ОСНОВНОЙ МОДУЛЬ
---
ОСНОВНОЙ МОДУЛЬ ПРЕДНАЗНАЧЕН СЧИТЫВАНИЯ ПОКАЗАНИЙ СО ВСЕХ СЕНСОРОВ ПРОЕКТА. ПОДГОТОВКИ ДАННЫХ ДЛЯ ОТОБРАЖЕНИЯ В ВИДЕ ГРАФИКОВ ЦИФРОВЫХ ПОКАЗАТЕЛЕЙ. ДЛЯ ФОРМИРОВАНИЯ 2D ГРАФИКОВ С СДАННЫМИ ПОСТУПАЮЩИМИ С МИКРОСХЕМЫ СЕНСОРА - ГИРОСКОПА, АКСЕЛЕРОМЕТРА, МАГНИТОМЕТРА

### Блок импорта библиотек необходимых для работы проекта

In [4]:
# Подключить библиотеки для мат обработки данных и генерации случайных данных
import numpy as np
import random
import math

# Подключить библиотеки для работы с 2D графиками
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator, NullFormatter

# Подключить библиотеки для работы анимацией
import matplotlib.animation as an

# Подключить библиотеки для работы с виджетами в Jupyter Lab
from IPython.display import display
import ipywidgets as widgets

# Подключить библиотеки для работы с завершающими функциями Jupyter Lab
import atexit

# Подключить библиотеки для работы с датой/временем
from datetime import datetime
import time
# Подключить библиотеки для асинхронной работы
import asyncio

# Подключить пользовательские модули
# ---
# Работа с интервалами в стиле setInterval JavaScript
from py.lib.lib_setinterval_rev01_v01 import setInterval
# Подключить модуль для работы с датчиком IMU ICM20948
from py.lib.lib_imu_20948_horizon_rev01_v03 import *
# Подключить модуль для работы с 2D фигурами
from py.lib.lib_figure2D_rev01_v01 import Figure2D

### Блок настроек среды исполнения Jupyter lab

In [3]:
# Задать режим работы отображения графической информации
%matplotlib  widget

# Инструкция требующая повторно перезагружать пользовательские функции после изменения их кода
%load_ext autoreload
%autoreload 2

### Блок настроек параметров 2D фигур и других других связанных с отображением

In [5]:
# Создать объект содержащий настройки графиков 'сырых данных Акселерометра
acc_opt_raw = {
                'figsize': (10, 8),
                'nrows': 3,
                'suptitle': 'ГРАФИК НЕ ОБРАБОТАННЫХ ДАННЫХ АКСЕЛЕРОМЕТРА',
                'abscissa': 'N',
                'ordinate': ('X', 'Y', 'Z'),
                'facecolor': ('lightslategray', 'lightslategray', 'lightslategray'),
                'alpha': (0.5, 0.5, 0.5),
                'graphcolor': ('green', 'blue', 'red'),
                'xlim': (0, 50),
                'ylim': (-2, 2) }

In [ ]:
# Создать объект содержащий настройки графиков усредненных данных Акселерометра
acc_opt_avg = {
                'figsize': (10, 8),
                'nrows': 3,
                'suptitle': 'ГРАФИК УСРЕДНЕННЫХ ДАННЫХ АКСЕЛЕРОМЕТРА',
                'abscissa': 'N',
                'ordinate': ('X', 'Y', 'Z'),
                'facecolor': ('lightslategray', 'lightslategray', 'lightslategray'),
                'alpha': (0.5, 0.5, 0.5),
                'graphcolor': ('green', 'blue', 'red'),
                'xlim': (0, 50),
                'ylim': (-2, 2) }

In [ ]:
# Создать объект содержащий настройки графиков 'сырых' данных Гироскопа
gyro_opt_raw = {
                'figsize': (10, 8),
                'nrows': 3,
                'suptitle': 'ГРАФИК НЕ ОБРАБОТАННЫХ ДАННЫХ ГИРОСКОПА',
                'abscissa': 'N',
                'ordinate': ('X', 'Y', 'Z'),
                'facecolor': ('lightslategray', 'lightslategray', 'lightslategray'),
                'alpha': (0.5, 0.5, 0.5),
                'graphcolor': ('green', 'blue', 'red'),
                'xlim': (0, 50),
                'ylim': (-500, 500) }

In [ ]:
# Создать объект содержащий настройки графиков усредненных данных Гироскопа
gyro_opt_avg = {
                'figsize': (10, 8),
                'nrows': 3,
                'suptitle': 'ГРАФИК УСРЕДНЕННЫХ ДАННЫХ ГИРОСКОПА',
                'abscissa': 'N',
                'ordinate': ('X', 'Y', 'Z'),
                'facecolor': ('lightslategray', 'lightslategray', 'lightslategray'),
                'alpha': (0.5, 0.5, 0.5),
                'graphcolor': ('green', 'blue', 'red'),
                'xlim': (0, 50),
                'ylim': (-500, 500) }

In [ ]:
# Создать объект содержащий настройки графиков 'сырых' данных Магнитометра
mag_opt_raw = {
                'figsize': (10, 8),
                'nrows': 3,
                'suptitle': 'ГРАФИК НЕ ОБРАБОТАННЫХ ДАННЫХ МАГНИТОМЕТРА',
                'abscissa': 'N',
                'ordinate': ('X', 'Y', 'Z'),
                'facecolor': ('lightslategray', 'lightslategray', 'lightslategray'),
                'alpha': (0.5, 0.5, 0.5),
                'graphcolor': ('green', 'blue', 'red'),
                'xlim': (0, 50),
                'ylim': (-500, 500) }

In [ ]:
# Создать объект содержащий настройки графиков усредненных данных Магнитометра
mag_opt_avg = {
                'figsize': (10, 8),
                'nrows': 3,
                'suptitle': 'ГРАФИК УСРЕДНЕННЫХ ДАННЫХ МАГНИТОМЕТРА',
                'abscissa': 'N',
                'ordinate': ('X', 'Y', 'Z'),
                'facecolor': ('lightslategray', 'lightslategray', 'lightslategray'),
                'alpha': (0.5, 0.5, 0.5),
                'graphcolor': ('green', 'blue', 'red'),
                'xlim': (0, 50),
                'ylim': (-500, 500) }

### Блок считывания данных с сенсоров: Акселерометра, Гироскопа, Магнитометра

In [4]:
# Подключить модуль для работы с датчиком IMU ICM20948
# from py.lib.lib_imu_20948_horizon_rev01_v03 import *

'''
    Класс чтения данных с трех сенсоров: Акселерометра, Гироскопа, Магнитометра.
'''
class ReadSensorIMU:
    def __init__(self, _opts) -> None:
        self.FigureOpt      = _opts            # хранит набор характеристик 2D фигуры
        self.ValueLimRaw1   = 4                # ограничитель - количество 'сырых' значений хранящихся в массивах AccArrRaw1/ GyroArrRaw1/ MagArrRaw2
        self.ValueLimRaw    = _opts['xlim'][1] # ограничитель - количество 'сырых' значений хранящихся в массивах AccArrRaw/ GyroArrRaw/ MagArrRaw
        self.ValueLimAvg    = _opts['xlim'][1] # ограничитель - количество усредненных значений хранящихся в массивах AccArrAvg/ GyroArrAvg/ MagAvgArr

        self.GyroFullScale = 1000 # величина настроенного диапазона Гироскопа (+/-) 1000 град/сек
        self.GyroSampleRate = 50  # частота семплирования Гироскопа - 50Hz
        self.GyroLowPass = 3      # фильтрация Гироскопа ~ 50Hz

        self.AccFullScale = 2     # величина настроенного диапазона Акселерометра (+/-) 2g
        self.AccSampleRate = 50   # частота семплирования Акселерометра - 50Hz
        self.AccLowPass = 3       # фильтрация Акселерометра ~ 50Hz

        self.MagSampleRate = 50   # частота семплирования Магнитометра - 50Hz

        self.IMU = ICM20948( gfs=self.GyroFullScale\
                            ,gsr=self.GyroSampleRate\
                            ,glp=self.GyroLowPass\
                            ,afs=self.AccFullScale\
                            ,asr=self.AccSampleRate
                            ,alp=self.AccLowPass\
                            ,msm=self.MagSampleRate) # инстанцировать модуль для работы с датчиком IMU ICM20948 

        self.AccRaw  = [0, 0, 0] # массив 'сырых', мгновенных значений  Акселерометра, по одному значению на каждую ось [X, Y, Z]
        self.GyroRaw = [0, 0, 0] # массив 'сырых', мгновенных значений  Гироскопа,     по одному значению на каждую ось [X, Y, Z]
        self.MagRaw  = [0, 0, 0] # массив 'сырых', мгновенных значений  Магнитометра,  по одному значению на каждую ось [X, Y, Z]

        self.AccAvg  = [0, 0, 0] # массив средних, мгновенных значений  Акселерометра, по одному значению на каждую ось [X, Y, Z]
        self.GyroAvg = [0, 0, 0] # массив средних, мгновенных значений  Гироскопа,     по одному значению на каждую ось [X, Y, Z]
        self.MagAvg  = [0, 0, 0] # массив средних, мгновенных значений  Магнитометра,  по одному значению на каждую ось [X, Y, Z]
        
        # Набор итоговых тройных массивов для хранения  'сырых' данных от трех (3) сенсоров, по трем (3) осям
        self.AccArrRaw  = [ [0]*self.ValueLimRaw, [0]*self.ValueLimRaw, [0]*self.ValueLimRaw ] # заполнить массив '0' значениями
        self.GyroArrRaw = [ [0]*self.ValueLimRaw, [0]*self.ValueLimRaw, [0]*self.ValueLimRaw ] # заполнить массив '0' значениями
        self.MagArrRaw  = [ [0]*self.ValueLimRaw, [0]*self.ValueLimRaw, [0]*self.ValueLimRaw ] # заполнить массив '0' значениями

        # Набор тройных укороченных (!) массивов для хранения 'сырых' данных от трех (3) сенсоров, по трем (3) осям
        # в соответствии со размером 'ValueLimRaw1' окна усреднения
        self.AccArrRawShort   = [ [0]*self.ValueLimRaw1, [0]*self.ValueLimRaw1, [0]*self.ValueLimRaw1 ] # заполнить массив '0' значениями
        self.GyroArrRawShort  = [ [0]*self.ValueLimRaw1, [0]*self.ValueLimRaw1, [0]*self.ValueLimRaw1 ] # заполнить массив '0' значениями
        self.MagArrRawShort   = [ [0]*self.ValueLimRaw1, [0]*self.ValueLimRaw1, [0]*self.ValueLimRaw1 ] # заполнить массив '0' значениям

        # Набор итоговых тройных массивов для хранения усредненных данных от трех (3) сенсоров, по трем (3) осям
        self.AccArrAvg   = [ [0]*self.ValueLimAvg, [0]*self.ValueLimAvg, [0]*self.ValueLimAvg ] # заполнить массив '0' значениями
        self.GyroArrAvg  = [ [0]*self.ValueLimAvg, [0]*self.ValueLimAvg, [0]*self.ValueLimAvg ] # заполнить массив '0' значениями
        self.MagArrAvg   = [ [0]*self.ValueLimAvg, [0]*self.ValueLimAvg, [0]*self.ValueLimAvg ] # заполнить массив '0' значениями

        self.TempRaw     = 0                    # Текущее значение температуры встроенного в ICM20948 термодатчика
        self.TempArrRaw  = [0]*self.ValueLimRaw # Массив текущих значений встроенного в ICM20948 термодатчика

        # Прочитать данные один раз - дать 'толчок' для постоянной работы датчика IMU
        self.IMU.read_magnetometer_data()
        self.IMU.read_accelerometer_gyro_data()
    
    '''
    
    '''
    def UpdateValue(self):

        # Получить'сырые' данные от всех датчиков IMU20948
        self.AccRaw, self.GyroRaw, self.MagRaw, self.TempRaw = self.IMU.read_acc_gyro_mag_data() # считать показания датчиков модуля ICM20948

        # Добавить одиночное показание Термодатчика в массив 'сырых' данных
        self.TempArrRaw.append(self.TempRaw)
        # Отслеживать размер массива 'сырых' данных
        if len(self.TempArrRaw) > self.ValueLimRaw:
            self.self.TempArrRaw.pop(0)
        
        # Добавить данные от трех датчиков по трем осям в массив 'сырых' данных и усредненных
        for i in range(3):
            # Добавить 'сырые' данные Акселерометра
            self.AccArrRaw[i].append(self.AccRaw[i])
            # Отслеживать размер массива 'сырых' данных
            if len(self.AccArrRaw[i]) > self.ValueLimRaw:
                self.self.AccArrRaw[i].pop(0)

            # Добавить 'сырые' данные Гироскопа
            self.GyroArrRaw[i].append(self.GyroRaw[i])
            # Отслеживать размер массива 'сырых' данных
            if len(self.GyroArrRaw[i]) > self.ValueLimRaw:
                self.self.GyroArrRaw[i].pop(0)
            
            # Добавить 'сырые' данные Магнитометра
            self.MagArrRaw[i].append(self.MagRaw[i])
            # Отслеживать размер массива 'сырых' данных
            if len(self.MagArrRaw[i]) > self.ValueLimRaw:
                self.self.MagArrRaw[i].pop(0)
            
            # Обновить усредненные данные по трем осям трех датчиков
            acc_avg  = sum( self.AccArrRaw[i][-self.ValueLimRaw1:]) / len( self.AccArrRaw[i][-self.ValueLimRaw1:])
            gyro_avg = sum(self.GyroArrRaw[i][-self.ValueLimRaw1:]) / len(self.GyroArrRaw[i][-self.ValueLimRaw1:])
            mag_avg  = sum( self.MagArrRaw[i][-self.ValueLimRaw1:]) / len( self.MagArrRaw[i][-self.ValueLimRaw1:])

            # Добавить усредненные данные Акселерометра
            self.AccArrAvg[i].append(acc_avg)
            # Отслеживать размер массива усредненных данных
            if len(self.AccArrAvg[i]) > self.ValueLimAvg:
                self.self.AccArrAvg[i].pop(0)
            
            # Добавить усредненные данные Гироскопа
            self.GyroArrAvg[i].append(gyro_avg)
            # Отслеживать размер массива усредненных данных
            if len(self.GyroArrAvg[i]) > self.ValueLimAvg:
                self.self.GyroArrAvg[i].pop(0)

            # Добавить усредненные данные Магнитометра
            self.MagArrAvg[i].append(mag_avg)
            # Отслеживать размер массива усредненных данных
            if len(self.MagArrAvg[i]) > self.ValueLimAvg:
                self.self.MagArrAvg[i].pop(0)

In [ ]:
def cleanup_imu():
    IMU.powerOff() # завершающий код для корректного выключения IMU
    print("Завершающая функция вызвана!")

# Регистрируем функцию cleanup_imu для вызова при завершении работы программы
atexit.register(cleanup_imu)

# Код пользователя
IMU = ReadSensorIMU( acc_opt_raw )

### Блок визуализации 'сырых' данных Акселерометра

In [ ]:
acc_grafics2D_raw = Figure2D( acc_opt_raw ) # инстанцировать объекта класса построения 2D фигур
acc_grafics2D_raw.CreateVectorFigure()      # Создать 'figure' matplotlib для построения тройного графика показаний датчика

acc_grafics2D_raw_anim = an.FuncAnimation( fig=acc_grafics2D_raw.Figure\
                                        ,func=acc_grafics2D_raw.UpdateAnimation\
                                        ,interval=200\
                                        ,frames=50) # frames=50, save_count=1500
# grafics_2D_anim = an.FuncAnimation(fig=grafics_2D.Figure, func=grafics_2D.UpdateGraph, interval=200, frames=50, save_count=1500) # сохраняет 1 500 кадров ?
plt.show()

#Codec = an.writers['ffmpeg']
#VideoParam = Codec(fps=15, bitrate=1800) #metadata={'artist': 'Me'}
#grafics_2D_anim.save('Line-Graph-Animation.mp4', VideoParam)



### Блок визуализации 'сырых' данных Гироскопа

In [ ]:
gyro_grafics2D_raw = Figure2D( gyro_opt_raw )
gyro_grafics2D_raw.CreateVectorFigure()

gyro_grafics2D_raw_anim = an.FuncAnimation(  fig=gyro_grafics2D_raw.Figure\
                                            ,func=gyro_grafics2D_raw.UpdateAnimation\
                                            ,interval=200\
                                            ,frames=50)
plt.show()

### Блок визуализации 'сырых' данных Магнитометра